# Funcionalidad principal de [banrep][pypi_banrep]
[pypi_banrep]: https://pypi.org/project/banrep/

## Requerimientos

### Importamos las otras librerías utilizadas en este ejemplo

In [ ]:
import numpy as np
import pandas as pd
import plotly_express as px
import plotly.offline as pyo
import spacy

### Y lo que usaremos de la librería [banrep][pypi_banrep]
[pypi_banrep]: https://pypi.org/project/banrep/

In [ ]:
from banrep.corpus import MiCorpus
from banrep.io import Textos, leer_palabras
from banrep.topicos import Topicos
from banrep.utils import crear_directorio

### Necesitamos un modelo NLP para español de [spaCy][spacy_models]
[spacy_models]: https://spacy.io/models

In [ ]:
nlp = spacy.load('es_core_news_md')

## Cargar listas de palabras y expresiones

A lo largo de un análisis de textos se suele requerir identificar palabras o expresiones presentes en ellos, bien sea para excluirlas del análisis (ej. *stopwords*), o porque su conteo es útil para generar indicadores (ej. *indicadores de sentimiento*), o porque el análisis se quiere limitar a estudiar aquellos textos que tienen cierto tipo de contenido (ej. *expresiones relacionadas con política económica*).

### Centralizar por proyecto

En cada proyecto se va a querer tener la flexibilidad para personalizar las palabras y expresiones que se quiere identificar. Para esto se recomienda usar hojas de excel, donde cada hoja puede contener categorías de palabras o expresiones. No es necesario usar un mismo archivo excel, pero facilita tener dichas listas de palabras centralizadas para abrir un solo archivo.

#### Uso de función: `leer_palabras`

Para este proyecto en particular, se quiere identificar (a) palabras *stopwords*, (b) palabras que identifican categorías de *sentimiento*, y (c) frases relacionadas con *incertidumbre en política económica*. Estas están en un archivo excel en las hojas `es_stops`, `es_emocion` y `es_epu` respectivamente.

Parámetros `archivo` y `hoja` determinan el archivo excel en disco, y la hoja a usar. (`rutalistas` es, en este caso, una variable que se pasa como argumento al parámetro `archivo`)

Parámetro `col_grupo` es el nombre de una columna en la hoja excel que determina el grupo al que pertenecen las palabras.

Parámetro `col_palabras` es el nombre de una columna en la hoja excel que contiene las palabras.

**Dado que estamos iterando un mismo archivo, este ejemplo asume que las columnas relevantes tienen el mismo nombre en cada hoja.**

In [ ]:
rutalistas = '~/Dropbox/datasets/wordlists/banrep.xlsx'
listas = dict()

hojas = ['es_stops', 'es_emocion', 'es_epu']
for hoja in hojas:
    listas[hoja] = leer_palabras(rutalistas, hoja, 
                                 col_grupo='type', col_palabras='word')

### Stopwords

Generalmente se quiere ignorar palabras comunes a todos los textos, llamadas *stopwords*, por no aportar al entendimiento de los diferentes textos.

In [ ]:
stopwords = listas['es_stops'].get("stopword")

print(f'{len(stopwords)} palabras stopwords.')

#### Ejemplo de stopwords en excel

![](img/stopwords.png)

### Pertenencia a listas de palabras predefinidas

Muchas veces se quiere contabilizar cuantas palabras de cada documento hacen parte de listas de palabras predefinidas. Por ejemplo, puedo tener listas de palabras "positivas" y "negativas", y querer contar cuantas palabras de los textos que voy a analizar hacen parte de estas listas. Esto sirve, por ejemplo, para crear indicadores de sentimiento basados en el conteo de palabras que pertenecen a emociones "contrarias".

In [ ]:
hoja = 'es_emocion'
wordlists = listas[hoja]

for tipo in wordlists:
    print(f'{len(wordlists.get(tipo))} palabras en grupo {tipo} de hoja {hoja}')

#### Ejemplo de emociones en excel

![](img/emocion.png)

### Pertenencia a listas de expresiones predefinidas

Al igual que con listas de palabras, a veces se quiere identificar frases de los textos en las que hay presencia de ciertas expresiones. Por ejemplo, un índice reconocido es el [Economic Uncertainty Index][epu], cuyos resultados se basan en el conteo de noticias en las que se encuentren diferentes expresiones relacionadas con incertidumbre en política económica. Por ejemplo, puedo querer identificar expresiones como *Banco de la República*, *déficit fiscal*, *política monetaria*, *inflación de alimentos*, *incertidumbre tributaria*, etc.

Identificar estas expresiones sirve también para proyectos en los que se quiere crear un [training set][prodigy] para entrenar un modelo de clasificación basado en *aprendizaje de máquina*, dado que el primer paso para dichos modelos es anotar una serie de frases que sean relevantes para lo que se quiere entrenar.


[epu]: http://www.policyuncertainty.com/research.html
[prodigy]: https://prodi.gy/

In [ ]:
hoja = 'es_epu'
express = listas[hoja]

for tipo in express:
    print(f'{len(express.get(tipo))} expresiones en grupo {tipo} de hoja {hoja}')

#### Ejemplo de expresiones en excel

![](img/epu.png)

## Preprocesamiento de texto y filtros

### Textos a usar

Se asume una carpeta en disco en la que hay archivos de texto. Si los textos que quiere utilizar están en archivos binarios como *.pdf*, *.docx*, etc., debe primero [extraer el texto][extraccion].

[extraccion]: https://munozbravo.github.io/banrep/uso_extraccion/

#### Uso de clase: `Textos`

Se usa para iterar los archivos de texto en disco.

Argumento `rutadatos` especifica ubicación de los textos en disco (parámetro `directorio`).

Parámetro `chars` filtra aquellas líneas de texto de cada archivo cuya longitud sea inferior al valor especificado. *69* es valor arbitrario que permite filtrar la mayoría de títulos y subtítulos.

Parámetro `parrafos` permite definir si se considera cada párrafo como un documento separado.

Parámetro `aleatorio` sirve cuando se quiere iterar los archivos aleatoriamente.

In [ ]:
rutadatos = '../datasets/'

datos = Textos(rutadatos, aleatorio=False, chars=69, parrafos=True)
print(datos)

### Filtros: palabras y tokens a ignorar

Se obtendrá información detallada de cada palabra gracias a [spaCy][web_spacy], lo que permite filtrar adicionalmente por criterios como [categoría gramatical][universal] (verbos, sustantivos, etc), si es algún tipo de [nombre propio][spacy_ents] (Juan, Colombia, Banco de la República), o si contiene caracteres que no hacen parte del alfabeto (números, monedas, etc). 

[web_spacy]: https://spacy.io/
[universal]: https://universaldependencies.org/es/index.html
[spacy_ents]: https://spacy.io/api/annotation#named-entities

En este caso elimino de cualquier análisis posterior todas las *stopwords* del archivo excel, las categorías gramaticales que identifican números, puntuación y símbolos, y aquellas "palabras" o tokens que contengan caracteres que no hacen parte del alfabeto.

In [ ]:
tags = ['NUM', 'PUNCT', 'SYM']

filtros = dict(stopwords=stopwords, postags=tags, entities=None, is_alpha=True)

## Corpus y sus estadísticas

### Crear el corpus

Un *corpus* es un conjunto de documentos, para el cual queremos obtener toda clase de estadísticas.

`MiCorpus` es la implementación de un corpus, el cual se inicializa con un modelo [spaCy][spacy_models] y un objeto `Textos`, y opcionalmente con los filtros especificados anteriormente, las listas de palabras que se quiere contar, y expresiones que se quiere encontrar. 

[spacy_models]: https://spacy.io/models

#### Uso de clase: `MiCorpus`

Se usa para inicializar el corpus. Es la estructura más importante de [banrep][pypi_banrep].

Argumento `nlp` es el modelo spaCy (parámetro `lang`).

Parámetro `datos` es el objeto Textos creado anteriormente en la variable *datos*.

Parámetros `filtros`, `wordlists` y `express` ya explicados anteriormente.

Parámetro `corta` sirve como filtro adicional, ignorando frases de pocos tokens (palabras, puntuación, símbolos, números, etc. En este ejemplo, frases con menos de 10 tokens).

[pypi_banrep]: https://pypi.org/project/banrep/

In [ ]:
corpus = MiCorpus(nlp, datos=datos, filtros=filtros, corta=9, 
                  wordlists=wordlists, express=express)
print(corpus)

### Estadísticas de corpus agregadas

In [ ]:
stats = corpus.corpus_stats()
stats.tail()

### Estadísticas desagregadas por token

In [ ]:
tokens = corpus.corpus_tokens()
tokens.tail()

In [ ]:
con_ngrams = corpus.corpus_ngramed()
con_ngrams.head(10)

### Estadísticas desagregadas por frase

In [ ]:
stats_frases = corpus.frases_stats()
stats_frases.head(10)

## Modelos de tópicos y su visualización

### Crear modelos LDA

Los modelos de tópicos se usan para encontrar "temáticas" subyacentes en los textos.

El parámetro básico a especificar en un modelo es el número de tópicos que se quiere considerar en el resultado.

Esta librería usa [Gensim][web_gensim] para la implementación del cálculo de los modelos. En su [documentación][gensim_tuts] encontrará todo lo necesario para correr este tipo de modelos y muchas técnicas adicionales no usadas en esta librería. [banrep][pypi_banrep] simplemente ofrece funciones para correr varios modelos LDA y seleccionar el mejor, todo basado en Gensim.

[web_gensim]: https://radimrehurek.com/gensim/models/ldamodel.html
[gensim_tuts]: https://radimrehurek.com/gensim/tutorial.html
[pypi_banrep]: https://pypi.org/project/banrep/

#### Uso de clase: `Topicos`

Se usa para crear modelos lda para diferentes números de tópicos.

Argumento `corpus` es una instancia de `MiCorpus`, y `n_topicos` es una lista de números para los cuales se quiere generar un modelo.

Parámetro `params` es un diccionario con parámetros que se usan en la [implementación LDA de Gensim][lda_gensim] (gensim.models.ldamodel.LdaModel).

Los modelos de tópicos suelen ser evaluados usando una medida llamada *Coherence Score*. Esta medida sugiere qué tan "interpretables" son los modelos. Un mayor score es un modelo más "interpretable", y por lo tanto mejor.

`Topicos` permite acceso a todos los modelos generados, y el mejor según Coherence Score se puede seleccionar usando el método `mejor_modelo`.  

[lda_gensim]: https://radimrehurek.com/gensim/models/ldamodel.html

In [ ]:
n_topicos = (5, 10, 15)
params = dict(passes=5, alpha='auto', eta='auto', random_state=100)

topicos = Topicos(corpus, n_topicos, params)
print(topicos)

In [ ]:
modelo = topicos.mejor_modelo()

### Visualización PyLDAvis (opcional)

En análisis de tópicos se suele usar PyLDAvis para visualizar resultados de un modelo.

La visualización es muy útil para entender la diferencia entre tópicos (distancia entre los círculos de la gráfica en costado izquierdo), prevalencia de cada tópico (tamaño de cada círculo), y el contenido de cada tópico (palabras más probables desplegadas en costado derecho para cada círculo).

El único "*pero*" es que suele generar advertencias de uso "obsoleto" (DeprecationWarning).

In [ ]:
import warnings

import pyLDAvis
import pyLDAvis.gensim

# Gráfica LDAvis de tópicos y sus palabras
bow = list(corpus)
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    vis = pyLDAvis.gensim.prepare(modelo, bow, corpus.id2word, sort_topics=False)

In [ ]:
pyLDAvis.display(vis)

### Alternativa a PyLDAvis (opcional)

Para entender diferencia entre tópicos también se puede usar un *heatmap*, en el que cada celda representa la "distancia" entre dos tópicos. 

Este, unido a las *Estadísticas de Modelo de tópicos* mencionadas abajo, provee la misma información que PyLDAvis.

In [ ]:
import plotly.figure_factory as ff

k = modelo.num_topics
diferencia, notas = modelo.diff(modelo, distance="hellinger", annotation=False)
anno_text = np.around(diferencia, decimals=2)

fig = ff.create_annotated_heatmap(z=diferencia, annotation_text=anno_text, 
                                  x=list(range(k)), y=list(range(k)), xgap=1, ygap=1,
                                  showscale=True, 
                                  )

pyo.iplot(fig)

## Estadísticas de Modelo de tópicos

### Distribución de probabilidad de palabras en cada tópico

Un tópico no es más que una serie de palabras con cierta probabilidad de ocurrir. Las palabras con mayor probabilidad son las que permiten "caracterizar" un tópico. 

#### Uso de método: `palabras_probables`

Genera un DataFrame con las palabras más probables de cada tópico en un modelo.

Parámetro `modelo` es un modelo LDA y `n` indica cuantas palabras se quiere incuir en el resultado.

In [ ]:
palabras = topicos.palabras_probables(modelo, n=20)

In [ ]:
palabras.head(10)

#### Visualización de palabras con gráficas de barras (opcional)

Visualizar las palabras más probables de cada tópico sirve para "ponerle nombre" a cada tópico.

Aunque esta información es fácilmente visible en `PyLDAvis`, estas son otras opciones.

In [ ]:
from plotly import tools
import plotly.graph_objs as go

k = modelo.num_topics
cols = 2
rows = int(np.ceil(k / cols))

subi = [(r+1, c+1) for r in range(rows) for c in range(cols)]

fig = tools.make_subplots(rows=rows, cols=cols, 
                          subplot_titles=([f'Tópico {t}' for t in range(k)]),
                          print_grid=False,
                         )

for i, t in enumerate(range(k)):
    dfg=palabras.loc[palabras['topico'] == t]
    dfg.sort_values(by='probabilidad', inplace=True)
    
    trace = go.Bar(x=dfg['probabilidad'], y=dfg['palabra'], orientation='h',)
    
    ix = subi[i]
    fig.add_trace(trace, row=ix[0], col=ix[1])

fig.layout.update(title='Principales palabras de cada tópico',
                  showlegend=False, yaxis=dict(automargin=True, ),
                  height=1800, width=1000)

pyo.iplot(fig)

#### Nubes de palabras (opcional)

Las nubes de palabra realmente no ofrecen valor adicional por encima de un gráfico de barras tradicional. Pero si le insisten que por favor genere nubes de palabras bonitas pero inútiles, esta sería una forma de hacerlo.

*Para controlar el tipo de letra puede buscar en internet uno que le guste. Este ejemplo usa [CabinSketch descargado de FONT Squirrel][cabin].*

[cabin]: https://www.fontsquirrel.com/fonts/cabinsketch

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

wc_params =dict(font_path="~/Dropbox/datasets/fonts/cabinsketch/CabinSketch-Bold.otf", 
                width=800, height=400, prefer_horizontal=0.6, background_color='white')

figwc = plt.figure(figsize=(16, 12))
figwc.subplots_adjust(hspace=0.05, wspace=0.1)

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    for t in range(k):
        freq = dict(modelo.show_topic(t, topn=20))
        wc = WordCloud(**wc_params).generate_from_frequencies(freq)

        plt.subplot(rows, cols, t+1).set_title(f"Tópico {t}")
        plt.plot()
        plt.imshow(wc, interpolation="bilinear")
        plt.axis("off")

    plt.tight_layout()
    figwc.show()

### Distribución de probabilidad de tópicos en documentos

Usualmente se quiere ver la probabilidad de cada tópico asociada a cada documento. Se puede  pensar un documento como generado de una distribución de probabilidad de tópicos. 

El método `stats_topicos` devuelve dicha distribución. Para cada documento (fila) muestra la  probabilidad de que hable de cada tópico (columna).

Basado en esta distribución, el método también devuelve la "prevalencia" de cada tópico: en cuantos documentos del corpus es dominante (el de mayor probabilidad).

#### Uso de método: `stats_topicos`

Parámetro `modelo` es un modelo LDA.

In [ ]:
doctopics, dominante = topicos.stats_topicos(modelo)

In [ ]:
doctopics.tail()

In [ ]:
dominante.head()

## Almacenamiento de objetos generados

### Criterios

El corpus y los modelos de tópicos generan estructuras que pueden ser útiles más adelante, y por lo tanto se quiere guardar a disco.

Se quiere usualmente almacenar modelos de "ngramas", el "diccionario" que contiene todas las palabras únicas existentes en el corpus, y el mejor modelo de tópicos opcionalmente con  su visualización.

#### Uso de función: `crear_directorio`

`crear_directorio` simplemente crea un directorio en disco, en el que se quiere guardar las estructuras generadas. En este caso se quiere crear directorio de salida para estructuras generales, y opcionalmente uno diferente para el mejor modelo.

In [ ]:
# Crear directorio de salida
dirsalida = crear_directorio('topicos')

# Crear directorio para mejor modelo.
dirmodelo = crear_directorio(dirsalida.joinpath(f'{topicos.top_k:0>2}'))

### Guardar n-gramas y diccionario generados en corpus

In [ ]:
#Guardar modelos de bigramas y trigramas
corpus.ngrams.get('bigrams').save(str(dirsalida.joinpath('bigrams')))
corpus.ngrams.get('trigrams').save(str(dirsalida.joinpath('trigrams')))

# Guardar diccionario
corpus.id2word.save(str(dirsalida.joinpath('id2word')))

### Guardar mejor modelo LDA y su visualización

In [ ]:
modelo.save(str(dirmodelo.joinpath('topicos.lda')))
pyLDAvis.save_html(vis, str(dirmodelo.joinpath('topicos.html')))